In [6]:
import numpy as np


# update import
%load_ext autoreload
%autoreload 2

In [27]:
# import msmd
from casatools import msmetadata 
from casatasks import gaincal, applycal, clearcal, delmod, ft, uvsub, split, concat, flagmanager, flagdata, tclean, hanningsmooth

# clear runtime directory
import os
import shutil

from eovsa_synop import wrap_wsclean, rotation_corr_util, split_by_scan, merge_scans, flag_ants
from suncasa.eovsa.eovsa_synoptic_imaging_pipeline import  trange2timerange
from suncasa.utils import helioimage2fits as hf
from astropy.io import fits
from astropy.wcs import WCS

# import Time
from astropy.time import Time
from glob import glob

In [28]:
from casatasks import flagdata, flagmanager, hanningsmooth

In [29]:
fname_root = "../UDB20241212.ms"

In [37]:
# flag and split by scan

flag_ants.flag_keep_antennas(fname_root)
split_by_scan.split_ms_by_scan(fname_root)

num_of_scans = len(glob(fname_root.split('.ms')[0]+"_scan*.ms"))

Flagging antennas: 13,14,15
Keeping antennas: 0,1,2,3,4,5,6,7,8,9,10,11,12
Flagging complete!
Found 4 scans in ../UDB20241212.ms
Processing scan 0
Created ../UDB20241212_scan0.ms
Processing scan 1
Created ../UDB20241212_scan1.ms
Processing scan 2
Created ../UDB20241212_scan2.ms
Processing scan 3
Created ../UDB20241212_scan3.ms


In [38]:

spws_all = [
  '0,1',
  '2,3,4',
  '5,6,7,8,9,10',
  '11,12,13,14,15,16,17,18,19,20',
  '21,22,23,24,25,26,27,28,29,30',
  '31,32,33,34,35,36,37,38,39,40',
  '41,42,43,44,45,46,47,48,49'
]


spws_this = spws_all[4]


for spws_this in spws_all:
    for scan_num in range(num_of_scans):

        msfile = fname_root.split('.ms')[0]+"_scan"+str(scan_num)+".ms"
        runtime_dir = "./runtime/"

        # get date from msfile name
        msmd = msmetadata()
        msmd.open(msfile)
        date_mjd = msmd.timerangeforobs(0)["begin"]["m0"]["value"]
        t_begin_mjd, t_end_mjd = msmd.timesforscans(scan_num)[0], msmd.timesforscans(scan_num)[-1]
        t_begin = Time(t_begin_mjd/3600/24, format='mjd')
        t_end = Time(t_end_mjd/3600/24, format='mjd')
        msmd.close()

        split_N1 = int(np.ceil((t_end-t_begin).sec/2000)) # split every 30-40 minutes
        split_N2 = 8 # sub split into 3-5 minutes 

        # if t_end-t_begin < 25 minutes, skip this scan
        if int((t_end-t_begin).sec/1500)  < 1:
            print("skip this scan")
            continue

        # convert MJD to date
        date_str = Time(date_mjd, format='mjd').iso
        date_withouttime = date_str[0:10]
        # use the date and 20:00 as reference time
        ref_time = Time(date_withouttime + " 20:00:00", format='iso')
        timerange = trange2timerange([t_begin, t_end])
        print(timerange)

        # step 0: flagging if flagverrsions file not exist
        if not os.path.exists(msfile+".flagversions"):
            flagdata(vis=msfile, mode="tfcrop", spw='', action='apply', display='',
                    timecutoff=2.0, freqcutoff=2.0, maxnpieces=2, flagbackup=False)
        
        # clean up
        if os.path.exists(runtime_dir):
            shutil.rmtree(runtime_dir)
        os.makedirs(runtime_dir)

        t_range_bins =  np.linspace(t_begin, t_end, split_N1+1)
        clean_obj = wrap_wsclean.WSClean(vis=msfile)



        # step 1 : make round1 image
        clean_obj.setup(size=1024, scale="2.5asec", weight_briggs=0.0, pol="xx", 
                        niter=3000, mgain=0.85, data_column="DATA",
                        name=runtime_dir+"eovsa", multiscale=True,
                        auto_mask=6, auto_threshold=3,
                        intervals_out=split_N1, 
                        no_negative=True, quiet=True,
                        spws=spws_this,)
        clean_obj.run(dryrun=False)

        # step 2: mutate the model images to the reftime
        # take all model images, and rotate them to reftime, and duplicate them N1*N2 times
        # step 2.1 : rotate the model images to reftime
        # if no model images, skip this scan


        # if split_N1 == 1, then there is only one model image, rename it to t0000
        if split_N1 == 1:
            fitsname = runtime_dir+"eovsa-t0000-model.fits"
            os.rename(runtime_dir+"eovsa-model.fits", fitsname)
            modelfitsfiles = [fitsname]
        else:
            modelfitsfiles = sorted(glob(runtime_dir+"eovsa-t*-model.fits"))

        if len(modelfitsfiles) == 0:
            print("no model images at scan", scan_num)
            continue

        for idx, fitsname in enumerate(modelfitsfiles):
            
            timerangethis  = trange2timerange([t_range_bins[idx], t_range_bins[idx+1]])
            heliofitsname = fitsname.replace("model.fits", "model.helio.fits")
            hf.imreg(vis=msfile, imagefile=fitsname, fitsfile=heliofitsname,
                    timerange=timerangethis)
            heliorotname = heliofitsname.replace("model.helio.fits", "model.helio.rot.fits")
            rotation_corr_util.solar_diff_rot_heliofits(heliofitsname, ref_time, 
                                                        heliorotname)
            
            rotation_corr_util.sunpyfits_to_j2000fits(heliorotname, heliorotname.replace(".fits", ".j2000.fits"),
                                                    template_fits=fitsname, overwrite_prev=True)


        # step 2.2 : duplicate the model images to N1*N2 times and rename them to t0000 to t0000+N1*N2-1
        # make new directory for the duplicated model images
        model_dir = runtime_dir + "modelrot/"
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)

        rotated_model_files = sorted(glob(runtime_dir + "*model.helio.rot.j2000.fits"))

        for i, model_file in enumerate(rotated_model_files):

            for j in range(split_N2):
                new_model_file = model_dir + "eovsa-t{:04d}-model.fits".format(i*split_N2+j)
                shutil.copy(model_file, new_model_file)

        # duplicate the model images to N1*N2 times and rename them to t0000 to t0000+N1*N2-1


        # predict 
        # step 3 : predict visibilities for each model image
        cmd  = "wsclean -predict -reorder -spws "+spws_this+" \
            -name runtime/modelrot/eovsa -intervals-out  "+ str(split_N1*split_N2) + ' ' + msfile
        os.system(cmd)


        # step 4 gaincal
        gaincal(vis=msfile, caltable=runtime_dir+"caltable", spw=spws_this, solint='inf', combine='scan',
                refant='0', gaintype='G', calmode='p', refantmode='flex', minsnr=1.0)
        applycal(vis=msfile, spw=spws_this, gaintable=runtime_dir+"caltable", interp='linear', calwt=False)



2024/12/12/16:03:54~2024/12/12/17:09:54
Running: wsclean -size 1024 1024 -scale 2.5asec -weight briggs 0.0 -niter 3000 -multiscale -mgain 0.85 -pol xx -auto-mask 6 -auto-threshold 3 -no-negative -intervals-out 2 -quiet -spws 0,1 -name ./runtime/eovsa ../UDB20241212_scan0.ms
Warning!!! Only one record exists in the ms ephemeris record.
           I have to use this only record to register the image.
Warning!!! Only one record exists in the ms ephemeris record.
           I have to use this only record to register the image.

WSClean version 3.5 (2024-07-30)
This software package is released under the GPL version 3.
Author: André Offringa (offringa@gmail.com).

=== IMAGING TABLE ===
       # Pol Ch JG ²G FG FI In Freq(MHz)
| Independent group:
+-+-J- 0  I   0  0  0  0  0  0  1095-1742 (52)
Counting number of scans... DONE (67)
Determining first and last row index... DONE (0-27200)
Reordering ../UDB20241212_scan0.ms into 2 x 1 parts.
Reordering: 0%....10%....20%....30%....40%....50%....60

2024-12-23 17:13:38	WARN	Calibrater::solve	Spectral window(s) 39, 40, 
2024-12-23 17:13:38	WARN	Calibrater::solve+	  could not be solved due to missing (pre-)calibration
2024-12-23 17:13:38	WARN	Calibrater::solve+	    in one or more of the specified tables.
2024-12-23 17:13:38	WARN	Calibrater::solve+	    Please check your results carefully!


The following MS spws have no corresponding cal spws in caltable: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 38 39 40 41 42 43 44 45 46 47 48 49 


2024-12-23 17:13:39	WARN	Calibrater::correct2 (VI2/VB2)	Spectral window(s) 38, 39, 40, 
2024-12-23 17:13:39	WARN	Calibrater::correct2 (VI2/VB2)+	  could not be corrected due to missing (pre-)calibration
2024-12-23 17:13:39	WARN	Calibrater::correct2 (VI2/VB2)+	    in one or more of the specified tables.
2024-12-23 17:13:39	WARN	Calibrater::correct2 (VI2/VB2)+	    Please check your results carefully!


2024/12/12/17:30:55~2024/12/12/18:29:55
Running: wsclean -size 1024 1024 -scale 2.5asec -weight briggs 0.0 -niter 3000 -multiscale -mgain 0.85 -pol xx -auto-mask 6 -auto-threshold 3 -no-negative -intervals-out 2 -quiet -spws 31,32,33,34,35,36,37,38,39,40 -name ./runtime/eovsa ../UDB20241212_scan1.ms
Warning!!! Only one record exists in the ms ephemeris record.
           I have to use this only record to register the image.
Warning!!! Only one record exists in the ms ephemeris record.
           I have to use this only record to register the image.

WSClean version 3.5 (2024-07-30)
This software package is released under the GPL version 3.
Author: André Offringa (offringa@gmail.com).

=== IMAGING TABLE ===
       # Pol Ch JG ²G FG FI In Freq(MHz)
| Independent group:
+-+-J- 0  I   0  0  0  0  0  0  11805-15055 (80)
Counting number of scans... DONE (60)
Determining first and last row index... DONE (0-20400)
Reordering ../UDB20241212_scan1.ms into 10 x 1 parts.
Reordering: 0%....10%....2

...

The following MS spws have no corresponding cal spws in caltable: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 39 40 41 42 43 44 45 46 47 48 49 


2024-12-23 17:14:31	WARN	Calibrater::correct2 (VI2/VB2)	Spectral window(s) 39, 40, 
2024-12-23 17:14:31	WARN	Calibrater::correct2 (VI2/VB2)+	  could not be corrected due to missing (pre-)calibration
2024-12-23 17:14:31	WARN	Calibrater::correct2 (VI2/VB2)+	    in one or more of the specified tables.
2024-12-23 17:14:31	WARN	Calibrater::correct2 (VI2/VB2)+	    Please check your results carefully!


2024/12/12/16:03:54~2024/12/12/17:09:54
Running: wsclean -size 1024 1024 -scale 2.5asec -weight briggs 0.0 -niter 3000 -multiscale -mgain 0.85 -pol xx -auto-mask 6 -auto-threshold 3 -no-negative -intervals-out 2 -quiet -spws 41,42,43,44,45,46,47,48,49 -name ./runtime/eovsa ../UDB20241212_scan0.ms
Warning!!! Only one record exists in the ms ephemeris record.
           I have to use this only record to register the image.


2024-12-23 17:14:38	SEVERE	image::_casaQuantityFromVar (file /tmp/build-via-sdist-pvbgv_tn/casatools-6.6.5.31/src/tools/image/image_cmpt.cc, line 6869)	Exception Reported: Exception: All selected pixels are masked.
2024-12-23 17:14:38	SEVERE	image::_casaQuantityFromVar (file /tmp/build-via-sdist-pvbgv_tn/casatools-6.6.5.31/src/tools/image/image_cmpt.cc, line 6869)+	... thrown by std::shared_ptr<casacore::ImageInterface<T>> casa::ImageRegridder<float>::_regrid() const [T = float] at File: /Users/casaci/bamboohome/xml-data/build-dir/CASA-CRBC665PLUS0-BPOSX14ARMPY310/casainstall/include/casacpp/imageanalysis/ImageAnalysis/ImageRegridder.tcc, line: 127


RuntimeError: Exception: All selected pixels are masked.
... thrown by std::shared_ptr<casacore::ImageInterface<T>> casa::ImageRegridder<float>::_regrid() const [T = float] at File: /Users/casaci/bamboohome/xml-data/build-dir/CASA-CRBC665PLUS0-BPOSX14ARMPY310/casainstall/include/casacpp/imageanalysis/ImageAnalysis/ImageRegridder.tcc, line: 127

In [35]:
spws_this

'21,22,23,24,25,26,27,28,29,30'

2024/12/15/18:39:54~2024/12/15/21:29:54


6

In [74]:
msfile

'./UDB20241215_scan2.ms'

Warning!!! Only one record exists in the ms ephemeris record.
           I have to use this only record to register the image.
Warning!!! Only one record exists in the ms ephemeris record.
           I have to use this only record to register the image.
Warning!!! Only one record exists in the ms ephemeris record.
           I have to use this only record to register the image.
Warning!!! Only one record exists in the ms ephemeris record.
           I have to use this only record to register the image.
Warning!!! Only one record exists in the ms ephemeris record.
           I have to use this only record to register the image.
Warning!!! Only one record exists in the ms ephemeris record.
           I have to use this only record to register the image.



WSClean version 3.5 (2024-07-30)
This software package is released under the GPL version 3.
Author: André Offringa (offringa@gmail.com).

=== IMAGING TABLE ===
       # Pol Ch JG ²G FG FI In Freq(MHz)
| Independent group:
+-+-J- 0  I   0  0  0  0  0  0  5305-8555 (80)
Counting number of scans... DONE (171)
Determining first and last row index... DONE (0-13600)
Reordering ./UDB20241215_scan2.ms into 10 x 1 parts.
Reordering: 0%....10%....20%....30%....40%....50%....60%....70%....80%....90%....100%
Initializing model visibilities: 0%....10%....20%....30%....40%....50%....60%....70%....80%....90%....100%
Reading runtime/modelrot/eovsa-t0000-model.fits...
Opening reordered part 0 spw 11 for ./UDB20241215_scan2.ms
Opening reordered part 1 spw 12 for ./UDB20241215_scan2.ms
Opening reordered part 2 spw 13 for ./UDB20241215_scan2.ms
Opening reordered part 3 spw 14 for ./UDB20241215_scan2.ms
Opening reordered part 4 spw 15 for ./UDB20241215_scan2.ms
Opening reordered part 5 spw 16 for ./UDB202

0

8 of 24 solutions flagged due to SNR < 1 in spw=11 at 2024/12/15/19:48:02.0
8 of 24 solutions flagged due to SNR < 1 in spw=12 at 2024/12/15/19:47:05.3
7 of 24 solutions flagged due to SNR < 1 in spw=13 at 2024/12/15/19:48:41.3
8 of 24 solutions flagged due to SNR < 1 in spw=14 at 2024/12/15/19:50:33.2
6 of 22 solutions flagged due to SNR < 1 in spw=15 at 2024/12/15/19:49:36.0
8 of 24 solutions flagged due to SNR < 1 in spw=16 at 2024/12/15/19:49:13.1
8 of 24 solutions flagged due to SNR < 1 in spw=17 at 2024/12/15/19:49:22.8
10 of 24 solutions flagged due to SNR < 1 in spw=18 at 2024/12/15/19:46:49.4
9 of 24 solutions flagged due to SNR < 1 in spw=19 at 2024/12/15/19:45:03.2
10 of 24 solutions flagged due to SNR < 1 in spw=20 at 2024/12/15/19:42:27.7


{'apply_tables': array([], dtype='<U5'),
 'selectvis': {'antennas': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
  'field': array([0]),
  'intents': array([], dtype='<U5'),
  'observation': array([0]),
  'scan': array([2]),
  'spw': array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20])},
 'solve_tables': array(['./runtime/caltable'], dtype='<U18'),
 'solvestats': {'above_minblperant': array([10, 10]),
  'above_minsnr': array([10, 10]),
  'data_unflagged': array([10, 10]),
  'expected': array([10, 10]),
  'spw0': {'above_minblperant': array([0, 0]),
   'above_minsnr': array([0, 0]),
   'ant0': {'above_minblperant': array([0, 0]),
    'above_minsnr': array([0, 0]),
    'data_unflagged': array([0, 0]),
    'expected': array([0, 0]),
    'used_as_refant': array([0])},
   'ant1': {'above_minblperant': array([0, 0]),
    'above_minsnr': array([0, 0]),
    'data_unflagged': array([0, 0]),
    'expected': array([0, 0]),
    'used_as_refant': array([0])},
   'ant10': {'abo

The following MS spws have no corresponding cal spws in caltable: 0 1 2 3 4 5 6 7 8 9 10 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 
